In [1]:
import os
import pickle
import argparse

import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB

import import_ipynb
from bohb_worker_1to3_2 import KerasWorker as worker

parser = argparse.ArgumentParser(description='bohb_server')
parser.add_argument('--min_budget',   type=float, help='Minimum number of epochs for training.',    default=50)
parser.add_argument('--max_budget',   type=float, help='Maximum number of epochs for training.',    default=200)
parser.add_argument('--n_iterations', type=int,   help='Number of iterations performed by the optimizer', default=100)
parser.add_argument('--worker', help='Flag to turn this into a worker process', action='store_true')
parser.add_argument('--run_id', type=str, help='A unique run id for this optimization run. An easy option is to use the job id of the clusters scheduler.')
parser.add_argument('--nic_name',type=str, help='Which network interface to use for communication.', default='lo')
parser.add_argument('--shared_directory',type=str, help='A directory that is accessible for all processes, e.g. a NFS share.', default='../BOHB_LAB_5CNN/result/2')

args=parser.parse_args(args=[])
    
# Every process has to lookup the hostname
host = hpns.nic_name_to_host(args.nic_name)


if args.worker:
    import time
    time.sleep(5)   # short artificial delay to make sure the nameserver is already running
    w = worker(run_id=args.run_id, host=host, timeout=120)
    w.load_nameserver_credentials(working_directory=args.shared_directory)
    w.run(background=False)
    exit(0)

# This example shows how to log live results. This is most useful
# for really long runs, where intermediate results could already be
# interesting. The core.result submodule contains the functionality to
# read the two generated files (results.json and configs.json) and
# create a Result object.
result_logger = hpres.json_result_logger(directory=args.shared_directory, overwrite=False)

# Start a nameserver:
NS = hpns.NameServer(run_id=args.run_id, host=host, port=1, working_directory=args.shared_directory)
ns_host, ns_port = NS.start()
# NS = hpns.NameServer(run_id=args.run_id, host='127.0.0.1', port=None, working_directory=args.shared_directory)
# ns_host, ns_port = NS.start()

# Start local worker
w = worker(run_id=args.run_id, host=host, nameserver=ns_host, nameserver_port=ns_port, timeout=120)
w.run(background=True)

# Run an optimizer
bohb = BOHB(  configspace = worker.get_configspace(),
                      run_id = args.run_id,
                      host=host,
                      nameserver=ns_host,
                      nameserver_port=ns_port,
                      result_logger=result_logger,
                      min_budget=args.min_budget, max_budget=args.max_budget,
               )
res = bohb.run(n_iterations=args.n_iterations)

# store results
with open(os.path.join(args.shared_directory, 'results.pkl'), 'wb') as fh:
    pickle.dump(res, fh)

# shutdown
bohb.shutdown(shutdown_workers=True)
NS.shutdown()

importing Jupyter notebook from bohb_worker_1to3_2.ipynb
(11, 504960, 2)
(11, 101280, 2)


DEBUG:hpbandster:wait_for_workers trying to get the condition
INFO:hpbandster:DISPATCHER: started the 'discover_worker' thread
DEBUG:hpbandster.run_None.worker.74c293875977.63381:WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x7f72d3c00cf8; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:1>
INFO:hpbandster:DISPATCHER: started the 'job_runner' thread
DEBUG:hpbandster.run_None.worker.74c293875977.63381:WORKER: No dispatcher found. Waiting for one to initiate contact.
INFO:hpbandster.run_None.worker.74c293875977.63381:WORKER: start listening for jobs
INFO:hpbandster:DISPATCHER: Pyro daemon running on 127.0.0.1:33019
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 0 currently in the pool.
INFO:hpbandster:DISPATCHER: discovered new worker, hpbandster.run_None.worker.74c293875977.63381140143758587712
DEBUG:hpbandster:HBMASTER: number of workers changed to 1
DEBUG:hpbandster:Enough workers to start this run!
DEBUG:hpband

(11572, 480, 2)
(2321, 480, 2)


DEBUG:hpbandster.run_None.worker.74c293875977.63381:WORKER: done with job (0, 0, 0), trying to register it.
INFO:hpbandster.run_None.worker.74c293875977.63381:WORKER: registered result for job (0, 0, 0) with dispatcher
DEBUG:hpbandster:DISPATCHER: job (0, 0, 0) finished
DEBUG:hpbandster:DISPATCHER: register_result: lock acquired
DEBUG:hpbandster:DISPATCHER: job (0, 0, 0) on hpbandster.run_None.worker.74c293875977.63381140143758587712 finished
DEBUG:hpbandster:job_id: (0, 0, 0)
kwargs: {'config': {'dropout_rate': 0.6660443989437261, 'lr': 0.024446129489084163, 'lstm_units': 172, 'num_conv_layers': 5, 'num_fc_layers': 1, 'num_fc_units_1': 131, 'num_filters_1': 6, 'optimizer': 'SGD', 'num_filters_2': 64, 'num_filters_3': 195, 'num_filters_4': 8, 'num_filters_5': 197, 'sgd_momentum': 0.8016771780206461}, 'budget': 66.66666666666666, 'working_directory': '.'}
result: {'loss': 0.04847908020019531, 'info': {'test accuracy': 0.9300947785377502, 'train accuracy': 0.999683141708374, 'validation 

DEBUG:hpbandster:job_callback for (0, 0, 2) got condition
DEBUG:hpbandster:Only 3 run(s) for budget 66.666667 available, need more than 17 -> can't build model!
DEBUG:hpbandster:HBMASTER: Trying to run another job!
DEBUG:hpbandster:job_callback for (0, 0, 2) finished
DEBUG:hpbandster:ITERATION: Advancing config (0, 0, 0) to next budget 200.000000
DEBUG:hpbandster:HBMASTER: schedule new run for iteration 0
DEBUG:hpbandster:HBMASTER: trying submitting job (0, 0, 0) to dispatcher
DEBUG:hpbandster:HBMASTER: submitting job (0, 0, 0) to dispatcher
DEBUG:hpbandster:DISPATCHER: trying to submit job (0, 0, 0)
DEBUG:hpbandster:DISPATCHER: trying to notify the job_runner thread.
DEBUG:hpbandster:HBMASTER: job (0, 0, 0) submitted to dispatcher
DEBUG:hpbandster:HBMASTER: running jobs: 1, queue sizes: (0, 1) -> wait
DEBUG:hpbandster:DISPATCHER: Trying to submit another job.
DEBUG:hpbandster:DISPATCHER: starting job (0, 0, 0) on hpbandster.run_None.worker.74c293875977.63381140143758587712
DEBUG:hpban

...

DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker 

DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker 

INFO:hpbandster.run_None.worker.74c293875977.63381:WORKER: start processing job (98, 0, 1)
DEBUG:hpbandster.run_None.worker.74c293875977.63381:WORKER: args: ()
DEBUG:hpbandster.run_None.worker.74c293875977.63381:WORKER: kwargs: {'config': {'dropout_rate': 0.595397061888355, 'lr': 0.018209683911922254, 'lstm_units': 256, 'num_conv_layers': 4, 'num_fc_layers': 1, 'num_fc_units_1': 8, 'num_filters_1': 27, 'optimizer': 'SGD', 'num_filters_2': 182, 'num_filters_3': 141, 'num_filters_4': 249, 'sgd_momentum': 0.5917385001548583}, 'budget': 66.66666666666666, 'working_directory': '.'}
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Start

DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potentia

DEBUG:hpbandster:DISPATCHER: job (99, 0, 1) dispatched on hpbandster.run_None.worker.74c293875977.63381140143758587712
DEBUG:hpbandster:DISPATCHER: jobs to submit = 0, number of idle workers = 0 -> waiting!
INFO:hpbandster.run_None.worker.74c293875977.63381:WORKER: start processing job (99, 0, 1)
DEBUG:hpbandster.run_None.worker.74c293875977.63381:WORKER: args: ()
DEBUG:hpbandster.run_None.worker.74c293875977.63381:WORKER: kwargs: {'config': {'dropout_rate': 0.8554659022962786, 'lr': 0.0029030354623405417, 'lstm_units': 151, 'num_conv_layers': 2, 'num_fc_layers': 2, 'num_fc_units_1': 228, 'num_filters_1': 165, 'optimizer': 'Adam', 'num_fc_units_2': 161, 'num_filters_2': 18}, 'budget': 200.0, 'working_directory': '.'}
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER: Found 1 potential workers, 1 currently in the pool.
DEBUG:hpbandster:DISPATCHER: Finished worker discovery
DEBUG:hpbandster:DISPATCHER: Starting worker discovery
DEBUG:hpbandster:DISPATCHER

In [2]:
# Step 5: Analysis
# Each optimizer returns a hpbandster.core.result.Result object.
# It holds informations about the optimization run like the incumbent (=best) configuration.
# For further details about the Result object, see its documentation.
# Here we simply print out the best config and some statistics about the performed runs.
id2config = res.get_id2config_mapping()
incumbent = res.get_incumbent_id()

print('Best found configuration:', id2config[incumbent]['config'])
print('A total of %i unique configurations where sampled.' % len(id2config.keys()))
print('A total of %i runs where executed.' % len(res.get_all_runs()))
print('Total budget corresponds to %.1f full function evaluations.'%(sum([r.budget for r in res.get_all_runs()])/args.max_budget))

Best found configuration: {'dropout_rate': 0.5447738322468123, 'lr': 0.03608870617327878, 'lstm_units': 192, 'num_conv_layers': 2, 'num_fc_layers': 1, 'num_fc_units_1': 14, 'num_filters_1': 27, 'optimizer': 'SGD', 'num_filters_2': 104, 'sgd_momentum': 0.5753198783260949}
A total of 250 unique configurations where sampled.
A total of 300 runs where executed.
Total budget corresponds to 200.0 full function evaluations.
